In [1]:
import numpy as np 
import math

In [2]:
def determineStepSize(x,d,r):
    n=len(d)
    stepSize=float("inf")
    for i in range(n):
        if d[i] < 0 and stepSize>-x[i]/d[i]:
            stepSize=-x[i]/d[i]
    stepSize=min([1,r*stepSize])
    return stepSize

In [3]:
def IPM(A,b,c,problem,r,epsilon,mu_0=10):
    import random
    m=len(A)
    n=len(A[0,:])

    x0=[]
    for i in range(n):
        x0.append([random.randint(1,100)])
    y0=[]
    for i in range(m):
        y0.append([random.randint(1,100)])
    s0=[]
    for i in range(n):
        #s0.append([random.randint(1,100)])
        s0.append([1])

    x0,y0,s0=np.array(x0),np.array(y0),np.array(s0)
    
    mu=[mu_0]

    if problem == "LP":
        e=np.ones((n,1))
    elif problem == "SOCP":
        e=np.ones((n,1))
    else:
        e=np.ones((n,1))

    k=0
    while mu[k]>epsilon*mu_0:
        mu[k]=mu_0*(1/10)**k
        x0=np.array(x0)
        y0=np.array(y0)
        s0=np.array(s0)
        solution=np.vstack((x0,y0,s0))
    
        if problem == "LP":
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
        elif problem == "SOCP":
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
        else:
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
        
        r1=b-A.dot(x0)
        r2=c-A.transpose().dot(y0)+s0
        r3=mu[k]*e-X.dot(S).dot(e)
        residual=np.vstack((r1,r2,r3))

        A1=np.hstack((A,np.zeros((m,m)),np.zeros((m,n))))
        A2=np.hstack((np.zeros((n,n)),A.transpose(),np.identity(n)))
        A3=np.hstack((S,np.zeros((n,m)),X))
        barA=np.vstack((A1,A2,A3))
    
    #NewtonDirection=np.linalg.inv(barA.transpose().dot(barA)).dot(barA.transpose()).dot(residual)
        NewtonDirection=np.linalg.inv(barA).dot(residual)
        delta_x=NewtonDirection[0:n,0].reshape(n,1)
        delta_y=NewtonDirection[n:n+m,0].reshape(m,1)
        delta_s=NewtonDirection[n+m:2*n+m,0].reshape(n,1)
    
        if problem == "LP":
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
        elif problem == "SOCP":
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
        else:
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
    
        x0=x0+stepSizeX*delta_x
        y0=y0+stepSizeS*delta_y
        s0=s0+stepSizeS*delta_s
        #mu[k+1]=(1/10)*x0.transpose().dot(s0)/4#complementary condition
        mu.append((1/10)*x0.transpose().dot(s0)/20)
        #print("mu is ",mu[k+1])
        #print("stepSizeX is ",stepSizeX)
        #print("stepSizeS is ",stepSizeS)
        #print('k=',k)
        #print(x0)
        #print(y0)
        #print(s0)
        k=k+1
    print("x0 is ", x0)
    r1=A.dot(x0)-b
    print("the finial cost is %f"% (c.transpose().dot(x0)))
    print("r1 is ",r1)

In [4]:
A=[[2,1,1,0],[1,3,0,1]]

b=[[70],[90]]
c=[[-40],[-60],[0],[0]]
A=np.array(A)
b=np.array(b)
c=np.array(c)

problem = "LP"
r=0.6
epsilon=1e-6
mu_0=10
IPM(A,b,c,problem,r,epsilon,mu_0)

x0 is  [[2.40153310e+01]
 [2.19986636e+01]
 [5.55326444e-10]
 [2.52201579e-10]]
the finial cost is -2280.533060
r1 is  [[0.02932571]
 [0.01132195]]


In [5]:
A=np.zeros((8,20))
b=np.zeros((8,1))
A[0,0],A[0,1],A[0,2],A[0,3],b[0,0]=1,1,1,1,400
A[1,4],A[1,5],A[1,6],A[1,7],b[1,0]=1,1,1,1,600
A[2,8],A[2,9],A[2,10],A[2,11],b[2,0]=1,1,1,1,400
A[3,12],A[3,13],A[3,14],A[3,15],b[3,0]=1,1,1,1,600
A[4,16],A[4,17],A[4,18],A[4,19],b[4,0]=1,1,1,1,1000

A[5,0],A[5,4],A[5,8],A[5,12],A[5,16],b[5,0]=1,1,1,1,1,600
A[6,1],A[6,5],A[6,9],A[6,13],A[6,17],b[6,0]=1,1,1,1,1,1000
A[7,2],A[7,6],A[7,10],A[7,14],A[7,18],b[7,0]=1,1,1,1,1,800

c=np.zeros((20,1))
c[0,0],c[1,0],c[2,0],c[3,0]=31,45,38,0
c[4,0],c[5,0],c[6,0],c[7,0]=29,41,35,0
c[8,0],c[9,0],c[10,0],c[11,0]=32,46,40,0
c[12,0],c[13,0],c[14,0],c[15,0]=28,42,600,0
c[16,0],c[17,0],c[18,0],c[19,0]=29,43,600,0

problem = "LP"
r=0.6
epsilon=1e-6
mu_0=10
IPM(A,b,c,problem,r,epsilon,mu_0)

x0 is  [[3.65440936e-11]
 [8.32547376e-11]
 [1.99997639e+02]
 [2.00000458e+02]
 [4.83020351e-11]
 [9.87288359e-04]
 [5.99994849e+02]
 [1.77810148e-11]
 [3.13558356e-12]
 [1.40572984e-12]
 [4.99903206e-12]
 [3.99995922e+02]
 [1.75989141e-01]
 [5.99818659e+02]
 [1.53623142e-15]
 [6.95067613e-12]
 [5.99819733e+02]
 [4.00168763e+02]
 [7.76673032e-16]
 [2.20722878e-11]]
the finial cost is 88399.110940
r1 is  [[-0.00190303]
 [-0.00416378]
 [-0.00407793]
 [-0.00535139]
 [-0.01150406]
 [-0.00427825]
 [-0.01158991]
 [-0.00751198]]


In [6]:
A=np.zeros((5,9))
b=np.zeros((5,1))
A[0,0],A[0,1],A[0,2],b[0,0]=1,1,1,70
A[1,3],A[1,4],A[1,5],b[1,0]=1,1,1,110
A[2,6],A[2,7],A[2,8],b[2,0]=1,1,1,80
A[3,0],A[3,3],A[3,6],b[3,0]=1,1,1,125
A[4,1],A[4,4],A[4,7],b[4,0]=1,1,1,60
c=np.zeros((9,1))
c[0,0],c[1,0],c[2,0]=162,121.5,82.8
c[3,0],c[4,0],c[5,0]=93.6,108,75
c[6,0],c[7,0],c[8,0]=158.4,100.8,100.8

problem = "LP"
r=0.6
epsilon=1e-6
mu_0=10
IPM(A,b,c,problem,r,epsilon,mu_0)

x0 is  [[1.31829502e-11]
 [1.88079136e-11]
 [7.00064207e+01]
 [1.10005714e+02]
 [1.51196270e-11]
 [5.74197981e-11]
 [1.49999411e+01]
 [6.00074810e+01]
 [4.99587660e+00]]
the finial cost is 25021.395570
r1 is  [[0.00642072]
 [0.00571385]
 [0.00329872]
 [0.00565495]
 [0.00748102]]


In [7]:
A1=np.array([[0,1,0],[0,1,0],[0,0,1],[0,0,1]])
A2=np.array([[0,-1,0],[0,0,0],[0,0,-1],[0,0,0]])
A3=np.array([[0,0,0],[0,-1,0],[0,0,0],[0,0,-1]])
A=np.hstack((A1,A2,A3))
print(A)

[[ 0  1  0  0 -1  0  0  0  0]
 [ 0  1  0  0  0  0  0 -1  0]
 [ 0  0  1  0  0 -1  0  0  0]
 [ 0  0  1  0  0  0  0  0 -1]]


In [8]:
b=np.zeros((4,1))
b[0]=1
b[1]=1/2
b[2]=0
b[3]=math.sqrt(3)/2
print(b)

[[1.       ]
 [0.5      ]
 [0.       ]
 [0.8660254]]


In [9]:
import numpy as np
import math
def functionJordanAlgebra(x):
    n=len(x[:,0])
    x0=x[0,0]
    x1=x[1:n,0]
    #print(x)
    #print(len())
    Jordan=np.hstack((x,np.vstack((x1.transpose(),x0*np.identity(n-1)))))
    return Jordan

A1=np.array([[0,1,0],[0,0,0],[0,0,1],[0,0,0]])
A2=np.array([[0,-1,0],[0,1,0],[0,0,-1],[0,0,1]])
A3=np.array([[0,0,0],[0,-1,0],[0,0,0],[0,0,-1]])
A=np.hstack((A1,A2,A3))

DataSet=[[0,0],[1,0],[1/2,math.sqrt(3)/2]]

c=np.zeros((9,1))
c[0]=1
c[3]=1
c[6]=1

x1=np.array([[0.57735],[0.5],[0.288675]])
x2=np.array([[0.57735],[0.5],[0.288675]])
x3=np.array([[0.57735],[0.5],[0.288675]])
x1=np.array([[200],[10],[10]])
x2=np.array([[200],[10],[10]])
x3=np.array([[200],[10],[10]])
x=np.vstack((x1,x2,x3))

y=np.zeros((4,1))

s1=np.array([[2],[1],[1]])
s2=np.array([[2],[1],[1]])
s3=np.array([[2],[1],[1]])
s=np.vstack((s1,s2,s3))

b=np.zeros((4,1))
b[0]=DataSet[1][0]-DataSet[0][0]
b[1]=DataSet[2][0]-DataSet[1][0]
b[2]=DataSet[1][1]-DataSet[0][1]
b[3]=DataSet[2][1]-DataSet[1][1]

e=np.zeros((3*3,1))
e[0]=1
e[3]=1
e[6]=1
mu=10
Tol=1e-8
k=1
while mu>Tol:
    if math.fmod(k,50)==0:
        print("k=%d" % k)
        print("mu=",mu[0,0])
        
    r1=b-A.dot(x)
    r2=c-A.transpose().dot(y)-s

    X1=functionJordanAlgebra(x1)
    X2=functionJordanAlgebra(x2)
    X3=functionJordanAlgebra(x3)
    X4=np.hstack((X1,np.zeros((3,3)),np.zeros((3,3))))
    X5=np.hstack((np.zeros((3,3)),X2,np.zeros((3,3))))
    X6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),X3))
    X=np.vstack((X4,X5,X6))

    S1=functionJordanAlgebra(s1)
    S2=functionJordanAlgebra(s2)
    S3=functionJordanAlgebra(s3)
    S4=np.hstack((S1,np.zeros((3,3)),np.zeros((3,3))))
    S5=np.hstack((np.zeros((3,3)),S2,np.zeros((3,3))))
    S6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),S3))
    S=np.vstack((S4,S5,S6))

    r3=mu*e-X.dot(S).dot(e)
    residual=np.vstack((r1,r2,r3))

    A4=np.hstack((A,np.zeros((4,4)),np.zeros((4,9))))
    A5=np.hstack((np.zeros((9,9)),A.transpose(),np.identity(9)))
    A6=np.hstack((S,np.zeros((9,4)),X))
    barA=np.vstack((A4,A5,A6))
    NewtoinDrection=np.linalg.inv(barA).dot(residual)

    deltaX1=NewtoinDrection[0:3,0].reshape(3,1)
    deltaX2=NewtoinDrection[3:6,0].reshape(3,1)
    deltaX3=NewtoinDrection[6:9,0].reshape(3,1)
    deltaY=NewtoinDrection[9:13,0].reshape(4,1)
    deltaS1=NewtoinDrection[13:16,0].reshape(3,1)
    deltaS2=NewtoinDrection[16:19,0].reshape(3,1)
    deltaS3=NewtoinDrection[19:22,0].reshape(3,1)
    
    stepSizeX1,stepSizeX2,stepSizeX3=0.025,0.025,0.025
    stepSizeS1,stepSizeS2,stepSizeS3=0.025,0.025,0.025
    stepSizeY=0.025
    
    x1=x1+stepSizeX1*deltaX1
    x2=x2+stepSizeX2*deltaX2
    x3=x3+stepSizeX3*deltaX3
    y=y+stepSizeY*deltaY
    s1=s1+stepSizeS1*deltaS1
    s2=s2+stepSizeS2*deltaS2
    s3=s3+stepSizeS3*deltaS3
    
    x=np.vstack((x1,x2,x3))
    s=np.vstack((s1,s2,s3))
    mu=x.transpose().dot(s)/9
    k=k+1
    #print("Feasibility:")
    #print(x1[0,0]-np.linalg.norm(x1[1:,0]))
    #print(x2[0,0]-np.linalg.norm(x2[1:,0]))
    #print(x3[0,0]-np.linalg.norm(x3[1:,0]))
    
    #print("Solution:")
    #for i in range(2):
    #    print(x1[i+1,0]+DataSet[0][i])
    #for i in range(2):
    #    print(x2[i+1,0]+DataSet[1][i])
    #for i in range(2):
    #    print(x3[i+1,0]+DataSet[2][i])
print("Feasibility:")
print(x1[0,0]-np.linalg.norm(x1[1:,0]))
print(x2[0,0]-np.linalg.norm(x2[1:,0]))
print(x3[0,0]-np.linalg.norm(x3[1:,0]))
    
print("Solution:")
for i in range(2):
    print(x1[i+1,0]+DataSet[0][i])
for i in range(2):
    print(x2[i+1,0]+DataSet[1][i])
for i in range(2):
    print(x3[i+1,0]+DataSet[2][i])

k=50
mu= 61.095808951390914
k=100
mu= 26.434327956070163
k=150
mu= 11.421037824931684
k=200
mu= 4.930561965149995
k=250
mu= 2.128021362285453
k=300
mu= 0.9183852303816818
k=350
mu= 0.3963378724047353
k=400
mu= 0.1710425438011604
k=450
mu= 0.07381460280597382
k=500
mu= 0.03185520180267583
k=550
mu= 0.013747332126068082
k=600
mu= 0.0059327560131264955
k=650
mu= 0.0025603217827668465
k=700
mu= 0.0011049245268862763
k=750
mu= 0.0004768378015302326
k=800
mu= 0.00020578264255623666
k=850
mu= 8.880691891771653e-05
k=900
mu= 3.832523846366195e-05
k=950
mu= 1.6539521032834762e-05
k=1000
mu= 7.1377443941293235e-06
k=1050
mu= 3.0803428307770965e-06
k=1100
mu= 1.3293431972489284e-06
k=1150
mu= 5.736872268162033e-07
k=1200
mu= 2.4757868010643554e-07
k=1250
mu= 1.068442872694438e-07
k=1300
mu= 4.610938909083768e-08
k=1350
mu= 1.9898825624290833e-08
Feasibility:
1.4984644081117438e-08
1.498474433425656e-08
1.498478086059407e-08
Solution:
0.5000000000001773
0.2886751345949902
0.5000000000001776
0.2886

In [10]:
#修改A,b
import numpy as np
import math
def functionJordanAlgebra(x):
    n=len(x[:,0])
    x0=x[0,0]
    x1=x[1:n,0]
    #print(x)
    #print(len())
    Jordan=np.hstack((x,np.vstack((x1.transpose(),x0*np.identity(n-1)))))
    return Jordan

A1=np.array([[0,1,0],[0,1,0],[0,0,1],[0,0,1]])
A2=np.array([[0,-1,0],[0,0,0],[0,0,-1],[0,0,0]])
A3=np.array([[0,0,0],[0,-1,0],[0,0,0],[0,0,-1]])
A=np.hstack((A1,A2,A3))

DataSet=[[0,0],[1,0],[1/2,math.sqrt(3)/2]]

c=np.zeros((9,1)) #權重
c[0]=1
c[3]=1
c[6]=1

x1=np.array([[0.57735],[0.5],[0.288675]])
x2=np.array([[0.57735],[0.5],[0.288675]])
x3=np.array([[0.57735],[0.5],[0.288675]])
x1=np.array([[200],[10],[10]])
x2=np.array([[200],[10],[10]])
x3=np.array([[200],[10],[10]])
x=np.vstack((x1,x2,x3))

y=np.zeros((4,1))

s1=np.array([[2],[1],[1]])
s2=np.array([[2],[1],[1]])
s3=np.array([[2],[1],[1]])
s=np.vstack((s1,s2,s3))

b=np.zeros((4,1))
b[0]=1
b[1]=1/2
b[2]=0
b[3]=math.sqrt(3)/2

e=np.zeros((3*3,1)) 
e[0]=1
e[3]=1
e[6]=1
mu=10
Tol=1e-8
k=1
while mu>Tol:
    if math.fmod(k,50)==0:
        print("k=%d" % k)
        print("mu=",mu[0,0])
        
    r1=b-A.dot(x)
    r2=c-A.transpose().dot(y)-s

    X1=functionJordanAlgebra(x1)
    X2=functionJordanAlgebra(x2)
    X3=functionJordanAlgebra(x3)
    X4=np.hstack((X1,np.zeros((3,3)),np.zeros((3,3))))
    X5=np.hstack((np.zeros((3,3)),X2,np.zeros((3,3))))
    X6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),X3))
    X=np.vstack((X4,X5,X6))

    S1=functionJordanAlgebra(s1)
    S2=functionJordanAlgebra(s2)
    S3=functionJordanAlgebra(s3)
    S4=np.hstack((S1,np.zeros((3,3)),np.zeros((3,3))))
    S5=np.hstack((np.zeros((3,3)),S2,np.zeros((3,3))))
    S6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),S3))
    S=np.vstack((S4,S5,S6))

    r3=mu*e-X.dot(S).dot(e)
    residual=np.vstack((r1,r2,r3))

    A4=np.hstack((A,np.zeros((4,4)),np.zeros((4,9))))
    A5=np.hstack((np.zeros((9,9)),A.transpose(),np.identity(9)))
    A6=np.hstack((S,np.zeros((9,4)),X))
    barA=np.vstack((A4,A5,A6))
    NewtoinDrection=np.linalg.inv(barA).dot(residual)

    deltaX1=NewtoinDrection[0:3,0].reshape(3,1)
    deltaX2=NewtoinDrection[3:6,0].reshape(3,1)
    deltaX3=NewtoinDrection[6:9,0].reshape(3,1)
    deltaY=NewtoinDrection[9:13,0].reshape(4,1)
    deltaS1=NewtoinDrection[13:16,0].reshape(3,1)
    deltaS2=NewtoinDrection[16:19,0].reshape(3,1)
    deltaS3=NewtoinDrection[19:22,0].reshape(3,1)
    
    stepSizeX1,stepSizeX2,stepSizeX3=0.025,0.025,0.025
    stepSizeS1,stepSizeS2,stepSizeS3=0.025,0.025,0.025
    stepSizeY=0.025
    
    x1=x1+stepSizeX1*deltaX1
    x2=x2+stepSizeX2*deltaX2
    x3=x3+stepSizeX3*deltaX3
    y=y+stepSizeY*deltaY
    s1=s1+stepSizeS1*deltaS1
    s2=s2+stepSizeS2*deltaS2
    s3=s3+stepSizeS3*deltaS3
    
    x=np.vstack((x1,x2,x3))
    s=np.vstack((s1,s2,s3))
    mu=x.transpose().dot(s)/9
    k=k+1
    #print("Feasibility:")
    #print(x1[0,0]-np.linalg.norm(x1[1:,0]))
    #print(x2[0,0]-np.linalg.norm(x2[1:,0]))
    #print(x3[0,0]-np.linalg.norm(x3[1:,0]))
    
    #print("Solution:")
    #for i in range(2):
    #    print(x1[i+1,0]+DataSet[0][i])
    #for i in range(2):
    #    print(x2[i+1,0]+DataSet[1][i])
    #for i in range(2):
    #    print(x3[i+1,0]+DataSet[2][i])
print("Feasibility:")
print(x1[0,0]-np.linalg.norm(x1[1:,0]))
print(x2[0,0]-np.linalg.norm(x2[1:,0]))
print(x3[0,0]-np.linalg.norm(x3[1:,0]))
    
print("Solution:")
for i in range(2):
    print(x1[i+1,0]+DataSet[0][i])
for i in range(2):
    print(x2[i+1,0]+DataSet[1][i])
for i in range(2):
    print(x3[i+1,0]+DataSet[2][i])

k=50
mu= 61.095808951390914
k=100
mu= 26.43432795607017
k=150
mu= 11.421037824931691
k=200
mu= 4.9305619651499955
k=250
mu= 2.128021362285454
k=300
mu= 0.9183852303816824
k=350
mu= 0.39633787240473567
k=400
mu= 0.17104254380116057
k=450
mu= 0.0738146028059739
k=500
mu= 0.03185520180267584
k=550
mu= 0.013747332126068094
k=600
mu= 0.005932756013126508
k=650
mu= 0.0025603217827668647
k=700
mu= 0.0011049245268862948
k=750
mu= 0.00047683780153028195
k=800
mu= 0.0002057826425562428
k=850
mu= 8.880691891772269e-05
k=900
mu= 3.832523846366811e-05
k=950
mu= 1.65395210328656e-05
k=1000
mu= 7.137744394141659e-06
k=1050
mu= 3.0803428307770965e-06
k=1100
mu= 1.3293431972489284e-06
k=1150
mu= 5.736872268223713e-07
k=1200
mu= 2.475786800694281e-07
k=1250
mu= 1.0684428723860427e-07
k=1300
mu= 4.610938905383024e-08
k=1350
mu= 1.9898825630458738e-08
Feasibility:
1.4984643970095135e-08
1.498474433425656e-08
1.498478086059407e-08
Solution:
0.5000000000001773
0.2886751345949902
0.5000000000001776
0.2886751

In [11]:
#def IPM(A,b,c,problem,r,epsilon,mu_0=10):
def IPM():
    A1=np.array([[0,1,0],[0,1,0],[0,0,1],[0,0,1]])
    A2=np.array([[0,-1,0],[0,0,0],[0,0,-1],[0,0,0]])
    A3=np.array([[0,0,0],[0,-1,0],[0,0,0],[0,0,-1]])
    A=np.hstack((A1,A2,A3))
    
    b=np.zeros((4,1))
    b[0]=1
    b[1]=1/2
    b[2]=0
    b[3]=math.sqrt(3)/2

    c=np.zeros((9,1)) 
    c[0]=1
    c[3]=1
    c[6]=1
    
    problem="SOCP"
    
    r=0
    epsilon=1e-6
    mu_0=10
    #print(A,"\n")
    #print(b,"\n")
    #print(c,"\n")
    #print(e,"\n")
    m=len(A)
    n=len(A[0,:])
    print(m,n)
    
    #if problem="LP":
    x0=[]
    for i in range(n):
        x0.append([10])
    y0=[]
    for i in range(m):
        y0.append([10])
    s0=[]
    for i in range(n):
        #s0.append([random.randint(1,100)])
        s0.append([1])
    #elif problem="SOCP":
        
    print(x0,"\n")
    print(y0,"\n")
    print(s0,"\n")
    
    x0,y0,s0=np.array(x0),np.array(y0),np.array(s0)
    mu=[mu_0]
    print(mu)
    
    if problem == "LP":
        e=np.ones((n,1))
    elif problem == "SOCP":
        e=np.ones((n,1))
        e=np.zeros((3*3,1)) 
        e[0]=1
        e[3]=1
        e[6]=1
    else:
        e=np.ones((n,1))
    print("\n",e)
    
    k=0
    while mu[k]>epsilon*mu_0:
        mu[k]=mu_0*(1/10)**k
        x0=np.array(x0)
        y0=np.array(y0)
        s0=np.array(s0)
        solution=np.vstack((x0,y0,s0))
        if problem == "LP":
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
        elif problem == "SOCP":
            X1=functionJordanAlgebra(x1)
            X2=functionJordanAlgebra(x2)
            X3=functionJordanAlgebra(x3)
            X4=np.hstack((X1,np.zeros((3,3)),np.zeros((3,3))))
            X5=np.hstack((np.zeros((3,3)),X2,np.zeros((3,3))))
            X6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),X3))
            X=np.vstack((X4,X5,X6))

            S1=functionJordanAlgebra(s1)
            S2=functionJordanAlgebra(s2)
            S3=functionJordanAlgebra(s3)
            S4=np.hstack((S1,np.zeros((3,3)),np.zeros((3,3))))
            S5=np.hstack((np.zeros((3,3)),S2,np.zeros((3,3))))
            S6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),S3))
            S=np.vstack((S4,S5,S6))
        else:
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
        r1=b-A.dot(x0)
        r2=c-A.transpose().dot(y0)+s0
        r3=mu[k]*e-X.dot(S).dot(e)
        residual=np.vstack((r1,r2,r3))

        A1=np.hstack((A,np.zeros((m,m)),np.zeros((m,n))))
        A2=np.hstack((np.zeros((n,n)),A.transpose(),np.identity(n)))
        A3=np.hstack((S,np.zeros((n,m)),X))
        barA=np.vstack((A1,A2,A3))
        #NewtonDirection=np.linalg.inv(barA.transpose().dot(barA)).dot(barA.transpose()).dot(residual)
        NewtonDirection=np.linalg.inv(barA).dot(residual)
        delta_x=NewtonDirection[0:n,0].reshape(n,1)
        delta_y=NewtonDirection[n:n+m,0].reshape(m,1)
        delta_s=NewtonDirection[n+m:2*n+m,0].reshape(n,1)
    
        if problem == "LP":
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
        elif problem == "SOCP":
            stepSizeX=0.025
            stepSizeS=0.025
        else:
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
    
        x0=x0+stepSizeX*delta_x
        y0=y0+stepSizeS*delta_y
        s0=s0+stepSizeS*delta_s
        #mu[k+1]=(1/10)*x0.transpose().dot(s0)/4#complementary condition
        mu.append((1/10)*x0.transpose().dot(s0)/9)
        #print("mu is ",mu[k+1])
        #print("stepSizeX is ",stepSizeX)
        #print("stepSizeS is ",stepSizeS)
        #print('k=',k)
        #print(x0)
        #print(y0)
        #print(s0)
        k=k+1
    print("x0 is ", x0)
    r1=A.dot(x0)-b
    print("the finial cost is %f"% (c.transpose().dot(x0)))
    print("r1 is ",r1)


In [12]:
IPM()

4 9
[[10], [10], [10], [10], [10], [10], [10], [10], [10]] 

[[10], [10], [10], [10]] 

[[1], [1], [1], [1], [1], [1], [1], [1], [1]] 

[10]

 [[1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
x0 is  [[-16.79877278]
 [ -9.64628577]
 [ -9.84695707]
 [ 18.05195862]
 [-10.59587253]
 [ -9.84695707]
 [ 30.80821164]
 [-10.12107915]
 [-10.66932332]]
the finial cost is 32.061397
r1 is  [[-0.05041324]
 [-0.02520662]
 [ 0.        ]
 [-0.04365915]]


In [24]:
def IPMTTT():
    #input
    A1=np.array([[0,1,0],[0,1,0],[0,0,1],[0,0,1]])
    A2=np.array([[0,-1,0],[0,0,0],[0,0,-1],[0,0,0]])
    A3=np.array([[0,0,0],[0,-1,0],[0,0,0],[0,0,-1]])
    A=np.hstack((A1,A2,A3))
    
    b=np.zeros((4,1))
    b[0]=1
    b[1]=1/2
    b[2]=0
    b[3]=math.sqrt(3)/2
    
    c=np.zeros((9,1))
    c[0]=1
    c[3]=1
    c[6]=1
    
    problem="SOCP"
    
    r=0
    epsilon=1e-6
    mu_0=10
    
    #print(A)
    #print("")
    #print(b)
    #print("")
    #print(c)
    #print("")
    #print(e)
    
    m=len(A)
    n=len(A[0,:])
    print(m,n)
    
    x0=[]
    for i in range(n):
        x0.append([10])
    y0=[]
    for i in range(m):
        y0.append([10])
    s0=[]
    for i in range(n):
        #s0.append([random.randint(1,100)])
        s0.append([1])
    
    print(x0)
    print("")
    print(y0)
    print("")
    print(s0)
    
    x0,y0,s0=np.array(x0),np.array(y0),np.array(s0)
    mu=[mu_0]
    print(mu)
    if problem == "LP":
        e=np.ones((n,1))
    elif problem == "SOCP":
        e=np.zeros((3*3,1))
        e[0]=1
        e[3]=1
        e[6]=1
    else:
        e=np.ones((n,1))
    print("\n",e)
    k=0
    while mu[k]>epsilon*mu_0:
        mu[k]=mu_0*(1/10)**k
        x0=np.array(x0)
        y0=np.array(y0)
        s0=np.array(s0)
        solution=np.vstack((x0,y0,s0))
        if problem == "LP":
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
        elif problem == "SOCP":
            X1=functionJordanAlgebra(x1)
            X2=functionJordanAlgebra(x2)
            X3=functionJordanAlgebra(x3)
            X4=np.hstack((X1,np.zeros((3,3)),np.zeros((3,3))))
            X5=np.hstack((np.zeros((3,3)),X2,np.zeros((3,3))))
            X6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),X3))
            X=np.vstack((X4,X5,X6))

            S1=functionJordanAlgebra(s1)
            S2=functionJordanAlgebra(s2)
            S3=functionJordanAlgebra(s3)
            S4=np.hstack((S1,np.zeros((3,3)),np.zeros((3,3))))
            S5=np.hstack((np.zeros((3,3)),S2,np.zeros((3,3))))
            S6=np.hstack((np.zeros((3,3)),np.zeros((3,3)),S3))
            S=np.vstack((S4,S5,S6))
        else:
            X=np.diag(x0[:,0])
            S=np.diag(s0[:,0])
          
        r1=b-A.dot(x0)
        r2=c-A.transpose().dot(y0)+s0
        r3=mu[k]*e-X.dot(S).dot(e)
        residual=np.vstack((r1,r2,r3))

        A1=np.hstack((A,np.zeros((m,m)),np.zeros((m,n))))
        A2=np.hstack((np.zeros((n,n)),A.transpose(),np.identity(n)))
        A3=np.hstack((S,np.zeros((n,m)),X))
        barA=np.vstack((A1,A2,A3))
        
        NewtonDirection=np.linalg.inv(barA).dot(residual)
        delta_x=NewtonDirection[0:n,0].reshape(n,1)
        delta_y=NewtonDirection[n:n+m,0].reshape(m,1)
        delta_s=NewtonDirection[n+m:2*n+m,0].reshape(n,1)
    
        if problem == "LP":
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
        elif problem == "SOCP":
            stepSizeX=0.025
            stepSizeS=0.025
        else:
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
    
        x0=x0+stepSizeX*delta_x
        y0=y0+stepSizeS*delta_y
        s0=s0+stepSizeS*delta_s
        mu.append((1/10)*x0.transpose().dot(s0)/9)
        k=k+1
        print("\n deltax is ", delta_x)
        print("\n x0 is ", x0)
    print("x0 is ", x0)
    r1=A.dot(x0)-b
    print("the finial cost is %f"% (c.transpose().dot(x0)))
    print("r1 is ",r1)
    
    

      print("\n x0 is ", x0)

IndentationError: unexpected indent (<ipython-input-24-07098d9b25c0>, line 136)

In [25]:
IPMTTT()

NameError: name 'IPMTTT' is not defined

In [14]:
 import random




   
    
    




        
        
        r1=b-A.dot(x0)
        r2=c-A.transpose().dot(y0)+s0
        r3=mu[k]*e-X.dot(S).dot(e)
        residual=np.vstack((r1,r2,r3))

        A1=np.hstack((A,np.zeros((m,m)),np.zeros((m,n))))
        A2=np.hstack((np.zeros((n,n)),A.transpose(),np.identity(n)))
        A3=np.hstack((S,np.zeros((n,m)),X))
        barA=np.vstack((A1,A2,A3))
    
    #NewtonDirection=np.linalg.inv(barA.transpose().dot(barA)).dot(barA.transpose()).dot(residual)
        NewtonDirection=np.linalg.inv(barA).dot(residual)
        delta_x=NewtonDirection[0:n,0].reshape(n,1)
        delta_y=NewtonDirection[n:n+m,0].reshape(m,1)
        delta_s=NewtonDirection[n+m:2*n+m,0].reshape(n,1)
    
        if problem == "LP":
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
        elif problem == "SOCP":
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
        else:
            stepSizeX=determineStepSize(x0,delta_x,r)
            stepSizeS=determineStepSize(s0,delta_s,r)
    
        x0=x0+stepSizeX*delta_x
        y0=y0+stepSizeS*delta_y
        s0=s0+stepSizeS*delta_s
        #mu[k+1]=(1/10)*x0.transpose().dot(s0)/4#complementary condition
        mu.append((1/10)*x0.transpose().dot(s0)/20)
        #print("mu is ",mu[k+1])
        #print("stepSizeX is ",stepSizeX)
        #print("stepSizeS is ",stepSizeS)
        #print('k=',k)
        #print(x0)
        #print(y0)
        #print(s0)
        k=k+1
    print("x0 is ", x0)
    r1=A.dot(x0)-b
    print("the finial cost is %f"% (c.transpose().dot(x0)))
    print("r1 is ",r1)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 54)